# HomeWork 3
En este módulo se ha realizado la evaluación de búsqueda por similitud semántica. <p>
Sin embargo, en este Homework se evaluará por **búsqueda vectorial**
- Script: [Homework Evaluation](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2025/03-evaluation/homework.md)

Obtenemos:
- documentos con id
- ground_truth data

In [1]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

Métricas de evaluación y función para evaluar cada documento.

In [ ]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)



In [9]:
def evaluacion_metricas(ground_truth,funcion_busqueda ): 
    relevantes= []
    print('Otención de resultados relevantes: [True, False, False, False, False], ...')
    for doc_groundt in tqdm(ground_truth):
        id_groundt= doc_groundt['document']
        resultados_search= funcion_busqueda(doc_groundt['question'], doc_groundt['course'] )
        
        relevante_registo = []

        for result in resultados_search:
            relevante_registo.append(id_groundt == result['id']  )
        # lo mismo: relevantes_registro = [result['id] == id_groundt for result in resultados_search]
        
        relevantes.append(relevante_registo)
    
    evaluacion_hr= hit_rate(relevantes)
    evaluacion_mrr= mrr(relevantes)


    return {'Evaluacion HR ': evaluacion_hr,  'Evaluacion MRR': evaluacion_mrr}

## Q1
Now let's evaluate our usual minsearch approach. What's the hitrate for this approach?
Indexamos según indican en el enunciado.

In [14]:
# indezxamos
import minsearch

#indexacion
index= minsearch.Index(
    text_fields=["question", "section", "text"], 
    keyword_fields=['course', 'id']
)

index.fit(documents)

In [19]:
# función búsqueda
def search_minsearch(query, course):
    resultados= index.search(
        query= query, 
        boost_dict = {'question': 1.5, 'section': 0.1},
        filter_dict= {"course": course} ,
        num_results= 5
    )

    return resultados


In [86]:
search_minsearch(question_20, "data-engineering-zoomcamp")

[{'text': 'You can set it up on your laptop or PC if you prefer to work locally from your laptop or PC.\nYou might face some challenges, especially for Windows users. If you face cnd2\nIf you prefer to work on the local machine, you may start with the week 1 Introduction to Docker and follow through.\nHowever, if you prefer to set up a virtual machine, you may start with these first:\nUsing GitHub Codespaces\nSetting up the environment on a cloudV Mcodespace\nI decided to work on a virtual machine because I have different laptops & PCs for my home & office, so I can work on this boot camp virtually anywhere.',
  'section': 'General course-related questions',
  'question': 'Environment - Should I use my local machine, GCP, or GitHub Codespaces for my environment?',
  'course': 'data-engineering-zoomcamp',
  'id': 'ddf6c1b3'},
 {'text': 'Choose the approach that aligns the most with your idea for the end project\nOne of those should suffice. However, BigQuery, which is part of GCP, will 

In [ ]:
print('MinSearch')
evaluacion_metricas(ground_truth, search_minsearch)

MinSearch
Otención de resultados relevantes: [True, False, False, False, False], ...


100%|██████████| 4627/4627 [00:15<00:00, 295.15it/s]


{'Evaluacion HR ': 0.848714069591528, 'Evaluacion MRR': 0.7288235717887772}

- **Hitrate**: 0.8487

## Q2. Vector search for question
Evaluate this seach method. What's MRR for it?

Embedding para las preguntas(questions) de documentos.
- Se generarán usando TF- IDF y  Singular Value Decomposition.
- Más detalles: [Build Your Own Search Engine](https://github.com/alexeygrigorev/build-your-own-search-engine)


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

# fit espera una lista de documentos
X = pipeline.fit_transform(texts) # se realiza el embedding de cada question


Usando VectorSearch de minsearch, se indexan las preguntas ya vectorizadas con los documentos.
Verificar última versión:
- `pip install -U minsearch qdrant_client`

In [35]:
from minsearch.vector import VectorSearch

vindex = VectorSearch(keyword_fields=['course'])
vindex.fit(X, documents)


In [41]:
type(X)

numpy.ndarray

Creamos la función de búsqueda vectorial

In [ ]:
def vector_search(query, course):
    
    
    return vindex.search(
        query, #questions embebidas, en este X
        filter_dict={'course': course}, 
        num_results=5
    )

def resultados_Vsearch(query, course): 
    # question= document['question']
    # course= document['course']
    # como ya se realizó fit_transform, ya se entrenó, entonces solo sería necesario transform, y como se espera una lista []
    question_embed= pipeline.transform([query])

    return vector_search(question_embed, course)

In [70]:
print('Vector Search')
evaluacion_metricas(ground_truth, resultados_Vsearch )

Vector Search
Otención de resultados relevantes: [True, False, False, False, False], ...


100%|██████████| 4627/4627 [00:07<00:00, 612.52it/s]


{'Evaluacion HR ': 0.48173762697212014, 'Evaluacion MRR': 0.3572833369353793}

- **MRR**: 0.357

## Q3. Vector Search for question and answer

In [71]:
# Creamos los embedding para question+answer
texts = []

for doc in documents:
    t = doc['question']  + ' ' + doc['text']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

# fit espera una lista de documentos
X = pipeline.fit_transform(texts) # se realiza el embedding de cada question

# Indexación
vindex = VectorSearch(keyword_fields=['course'])
vindex.fit(X, documents)


In [72]:
print('Vector Search (question+answer)')
evaluacion_metricas(ground_truth, resultados_Vsearch )

Vector Search (question+answer)
Otención de resultados relevantes: [True, False, False, False, False], ...


100%|██████████| 4627/4627 [00:07<00:00, 598.74it/s]


{'Evaluacion HR ': 0.8210503566025502, 'Evaluacion MRR': 0.6717347453353508}

- **HR**: 0.82

NOTA: Vemos un claro incremento de aproximadamente el doble en ambas métricas, cuando se añade las respuestas y no solo se usa las preguntas

## Q4. Qdrant
Evaluación con las siguientes configuraciones:
- text = doc['question'] + ' ' + doc['text']
- model_handle = "jinaai/jina-embeddings-v2-small-en"
- limit = 5

Más detalles en: 
- Setup: [set_up_qdrant](https://github.com/Halsey26/llm-zoomcamp/blob/main/02_Mod_vector_search/set_up_qdrant.md)
- [Vector_search_qdrant](https://github.com/Halsey26/llm-zoomcamp/blob/main/02_Mod_vector_search/vector_search_qdrant.ipynb)

In [74]:
from qdrant_client import QdrantClient, models

# Inicializamos el cliente
cliente = QdrantClient("http://localhost:6333")

modelo = "jinaai/jina-embeddings-v2-small-en"
dimension= 512



In [ ]:
# coleccion

nombre_coleccion= 'homework3'
cliente.create_collection(
    collection_name= nombre_coleccion, 
    vectors_config=models.VectorParams(
        size= dimension, 
        distance= models.Distance.COSINE
    )
)

# creacion de los Puntos
# para almacenar los puntos
points= []
id=0 #por defecto el identificador de los puntos 0 


for doc in documents:
    point = models.PointStruct(
        id=id, 
        vector = models.Document(text= doc['question'] + ' ' + doc['text'], model= modelo),
        payload= {"text": doc['text'], "section": doc['section'], "course":doc["course"], "id": doc['id']}
    )
    points.append(point)
    id +=1

# Insercción de los puntos en la Colección
cliente.upsert(
    collection_name= nombre_coleccion, 
    points= points
)


Fetching 5 files: 100%|██████████| 5/5 [00:07<00:00,  1.57s/it]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

Función búsqueda usando Qdrant

In [84]:
def search_qdrant(query, limit= 5):
    results = cliente.query_points(
        collection_name= nombre_coleccion,
        query = models.Document( #realiza el embedding de la query localmente con el modelo elegido
            text= query, 
            model = modelo
        ),
        limit = limit, # cantidad de respuestas
        with_payload=True  # para obtener la metada en los resultados 
    )
    return results


Necesitamos una estructura <p>
Estrutura de los resultados esperados:
```
[{'text': ' ',
  'section': '',
  'question': '',
  'course': 'data-engineering-zoomcamp',
  'id': ''} 
  {...},
```

In [83]:
question_20 = documents[20]['question']
question_20

'Environment - Should I use my local machine, GCP, or GitHub Codespaces for my environment?'

In [85]:
resultados= search_qdrant(question_20)
resultados

QueryResponse(points=[ScoredPoint(id=20, version=0, score=0.9360642, payload={'text': 'You can set it up on your laptop or PC if you prefer to work locally from your laptop or PC.\nYou might face some challenges, especially for Windows users. If you face cnd2\nIf you prefer to work on the local machine, you may start with the week 1 Introduction to Docker and follow through.\nHowever, if you prefer to set up a virtual machine, you may start with these first:\nUsing GitHub Codespaces\nSetting up the environment on a cloudV Mcodespace\nI decided to work on a virtual machine because I have different laptops & PCs for my home & office, so I can work on this boot camp virtually anywhere.', 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp', 'id': 'ddf6c1b3'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=23, version=0, score=0.9106379, payload={'text': 'Choose the approach that aligns the most with your idea for the end project\nOne of thos